In [239]:
### Tarefas:
### Faixa de preço praticada pelos concorrentes
### Como é o fluxo de pessoas nesses locais, 
### qual é a população e a densidade demográfica dos bairros onde os concorrentes estão...
sc

<SparkContext master=local[*] appName=PySparkShell>

In [437]:
from pyspark.sql import Row
from pyspark import SparkContext

In [562]:
def extrair_dados(linha):
    linha = linha.replace(', ', ' - ')
    linha = linha.replace('"', '')
    return linha.split(',')

In [563]:
dBairro = sc.textFile('data/bairros.csv')
dBairro = dBairro.map(extrair_dados)
hBairro = dBairro.first()
bairro =  dBairro.filter(lambda dBairro: dBairro != hBairro)

In [564]:
dEventoFluxo = sc.textFile('data/eventos_de_fluxo.csv')
dEventoFluxo = dEventoFluxo.map(extrair_dados)
hEventoFluxo = dEventoFluxo.first()
eventoFluxo = dEventoFluxo.filter(lambda dEventoFluxo: dEventoFluxo != hEventoFluxo)

In [755]:
dConcorrente = sc.textFile('data/concorrentes.csv')
dConcorrente = dConcorrente.map(extrair_dados)
hConcorrente = dConcorrente.first()
concorrente = dConcorrente.filter(lambda dConcorrente: dConcorrente != hConcorrente)

In [756]:
populacao = sqlContext.read.json("data/populacao.json")
populacao.printSchema
populacao.printSchema()
populacao.registerTempTable("populacao")
sqlContext.sql("select * from populacao")

root
 |-- codigo: long (nullable = true)
 |-- populacao: long (nullable = true)



DataFrame[codigo: bigint, populacao: bigint]

### Qual é a faixa de preço praticada pelos concorrentes ? 

In [796]:
concorrente.take(33)

[['431962533652067',
  'Boizão Lanches',
  'Bar',
  '2',
  '13190-000 Monte Mor',
  'Monte Mor',
  'SP',
  ''],
 ['1663855903830869',
  'Bar do Serjão',
  'Bar',
  '0',
  'Rua das Dracenas - Americana',
  'Americana',
  'SP',
  ''],
 ['567824576564110',
  'Recanto Do Kuca',
  'Restaurant',
  '0',
  'Jarinu',
  'Jarinu',
  'SP',
  ''],
 ['202740866540615',
  'Dedé Abelhuda',
  'Grocery Store',
  '0',
  'SP - 13150000 Cosmópolis',
  'Cosmópolis',
  'SP',
  ''],
 ['1784900838394305',
  'Tenshi Sushi Boteco Itu',
  'Sushi Restaurant',
  '3',
  'Av. Plaza - 170 - 13302-100 Itu',
  'Itu',
  'SP',
  ''],
 ['1539938616222537',
  'Delicias de mãe',
  'Restaurant - Other',
  '1',
  'Itatiba',
  'Itatiba',
  'SP',
  ''],
 ['315940385145609',
  'ChokoLipe',
  'Grocery Store',
  '0',
  '13315000 Cabreúva',
  'Cabreúva',
  'SP',
  ''],
 ['294588290644303',
  'O RANCHOo',
  'Pub',
  '0',
  'Um dilema. - 13253-232 Itatiba',
  'Itatiba',
  'SP',
  ''],
 ['858228024202335',
  'Kentinhas Já',
  'Grocery 

In [758]:
def converter_tipo(valor, padrao, *tipos):
    for t in tipos:
        try:
            return t(valor)
        except ValueError:
            continue
    return padrao

In [794]:
hConcorrente.index('codigo_bairro')

7

In [788]:
index_categoria = hConcorrente.index('categoria')
index_codigo = hConcorrente.index('codigo')
index_faixa_preco = hConcorrente.index('faixa_preco')
index_codigo_bairro = hConcorrente.index('codigo_bairro')

In [1000]:
rdd_concorrente = concorrente.map(lambda x: (x[index_categoria].split(' - '), 
                                            converter_tipo(x[index_faixa_preco], 0, int, float),
                                            x[index_codigo_bairro]))

In [1001]:
concorrente_faixa_preco = rdd_concorrente.sortBy(lambda x: x[0], ascending=True).collect()

In [1002]:
#rdd_concorrente_categ = rdd_concorrente.groupByKey()
#rdd_concorrente_categ.take(10)

In [1003]:
concorrente_faixa_preco

[(['American Restaurant', 'Bar'], 0, ''),
 (['American Restaurant', 'Grocery Store'], 2, ''),
 (['American Restaurant', 'Mexican Restaurant', 'Burger Restaurant'],
  2,
  '355620111'),
 (['American Restaurant', 'Tex-Mex Restaurant', 'Grocery Store'], 1, ''),
 (['Arabian Restaurant'], 0, ''),
 (['Argentinian Restaurant'], 0, ''),
 (['Argentinian Restaurant', 'Bar'], 4, ''),
 (['Asian Restaurant'], 2, ''),
 (['Asian Restaurant'], 1, '3552403003'),
 (['Asian Restaurant', 'Sushi Restaurant', 'Grocery Store'], 2, ''),
 (['Bakery'], 1, ''),
 (['Bakery'], 3, ''),
 (['Bakery'], 3, ''),
 (['Bakery'], 2, ''),
 (['Bakery'], 2, ''),
 (['Bakery'], 1, ''),
 (['Bakery'], 0, ''),
 (['Bakery'], 0, ''),
 (['Bakery'], 0, ''),
 (['Bakery'], 0, ''),
 (['Bakery'], 0, ''),
 (['Bakery'], 0, ''),
 (['Bakery'], 1, '35095089'),
 (['Bakery'], 3, '35095069'),
 (['Bakery', 'Brazilian Restaurant'], 4, ''),
 (['Bakery', 'Brazilian Restaurant', 'Fast Food Restaurant'], 0, ''),
 (['Bakery', 'Cafe'], 0, '35095071'),
 ([

In [1004]:
concorrente_categoria = {}
faixa_preco = []
for concor in concorrente_faixa_preco:
    for categoria in concor[0]:
        if not concorrente_categoria.get(categoria):
            faixa_preco = []
        concorrente_categoria[categoria] = {}
        concorrente_categoria[categoria]['codigo_bairro'] = concor[2]
        faixa_preco.append(concor[1])
        concorrente_categoria[categoria]['preco_min'] = min(faixa_preco)
        concorrente_categoria[categoria]['preco_max'] = max(faixa_preco)
   
concorrente_categoria

{'Accessories': {'codigo_bairro': '', 'preco_max': 2, 'preco_min': 2},
 'Adult Entertainment Service': {'codigo_bairro': '35095064',
  'preco_max': 4,
  'preco_min': 1},
 'Agricultural Service': {'codigo_bairro': '355670114',
  'preco_max': 3,
  'preco_min': 0},
 'Agriculture Company': {'codigo_bairro': '3536505004',
  'preco_max': 0,
  'preco_min': 0},
 'Airport': {'codigo_bairro': '', 'preco_max': 2, 'preco_min': 2},
 'Alternative & Holistic Health Service': {'codigo_bairro': '',
  'preco_max': 0,
  'preco_min': 0},
 'American Restaurant': {'codigo_bairro': '35095068',
  'preco_max': 4,
  'preco_min': 0},
 'Amusement & Theme Park': {'codigo_bairro': '',
  'preco_max': 2,
  'preco_min': 1},
 'Antique Store': {'codigo_bairro': '35095084',
  'preco_max': 2,
  'preco_min': 2},
 'Apartment & Condo Building': {'codigo_bairro': '355670117',
  'preco_max': 1,
  'preco_min': 1},
 'Apparel & Clothing': {'codigo_bairro': '', 'preco_max': 0, 'preco_min': 0},
 'Appliances': {'codigo_bairro': '', 

### Como é o fluxo de pessoas nesses locais ? 
#### Arquivo populacao.csv para relacionar com o código do bairro obter quantidade de habitantes por bairro.

In [ ]:
rdd_bairro = bairro.map(lambda x: Row(codigo=x[0], nome=x[1], municipio=x[2], uf=x[3], area=x[4]))

In [968]:
rdd_bairro.take(1)

[Row(area='68.0009', codigo='355620110', municipio='Valinhos', nome='Observatório', uf='SP')]

In [969]:
df_bairro = sqlContext.createDataFrame(rdd_bairro)

In [980]:
df_bairro.registerTempTable("bairro")
df_populacao_bairro = sqlContext.sql("select * from bairro b left join populacao p where b.codigo = p.codigo")

In [981]:
df_populacao_bairro.show()

+---------+----------+-----------+--------------------+---+----------+---------+
|     area|    codigo|  municipio|                nome| uf|    codigo|populacao|
+---------+----------+-----------+--------------------+---+----------+---------+
|  68.0009| 355620110|   Valinhos|        Observatório| SP| 355620110|     8717|
| 0.981768|3519071024|Hortolândia|             Rp 6-24| SP|3519071024|     5764|
| 0.808537|3536505002|   Paulínia|   Jardim De Itapoan| SP|3536505002|     1195|
|  2.21108|3519071026|Hortolândia|             Rp 6-26| SP|3519071026|    17840|
| 0.386199|3536505001|   Paulínia|       Nova Paulínia| SP|3536505001|     1252|
|  1.70656|3536505018|   Paulínia|  Balneário Tropical| SP|3536505018|      206|
|  1.84108|3536505019|   Paulínia|         Nova Veneza| SP|3536505019|     1809|
|  47.0085|  35095090|   Campinas|        Campo Grande| SP|  35095090|   105720|
|   3.1602|3536505021|   Paulínia|      Jardim Itapoan| SP|3536505021|     2334|
|  0.12222|3536505024|   Pau

In [1038]:
rdd_eventoFluxo = eventoFluxo.map(lambda x: Row(codigo=x[0], datetime=x[1], codigo_concorrente=x[2]))

In [1039]:
rdd_eventoFluxo.take(3)

[Row(codigo='oMn07h1bJYV0Wdx+RTzsDcT8JQlT7QXc7q8A/4y+cO5gBQKOUoYGBEn1EKY7LIg4', codigo_concorrente='650509405109544', datetime='2017-07-27 09:51:02'),
 Row(codigo='iZuQeTd9am+qfaiqnn5kkixogIbwN0nY2gtMwZqH9bFqph8qYvYchFRokjl3Pe/V', codigo_concorrente='650509405109544', datetime='2017-06-24 14:00:26.405'),
 Row(codigo='iIMvwWSnQoW0aqQlcvjc8A6LvjkRX1HLppdkdQZapPVVv7VJLDVGqzqsaBrpuu+7', codigo_concorrente='650509405109544', datetime='2017-07-06 21:51:11.056')]

In [1040]:
df_eventoFluxo = sqlContext.createDataFrame(rdd_eventoFluxo)

In [1041]:
df_eventoFluxo.registerTempTable("evento_fluxo")
df_eventoFluxo = sqlContext.sql("select * from evento_fluxo")

In [1042]:
df_eventoFluxo.show()

+--------------------+------------------+--------------------+
|              codigo|codigo_concorrente|            datetime|
+--------------------+------------------+--------------------+
|oMn07h1bJYV0Wdx+R...|   650509405109544| 2017-07-27 09:51:02|
|iZuQeTd9am+qfaiqn...|   650509405109544|2017-06-24 14:00:...|
|iIMvwWSnQoW0aqQlc...|   650509405109544|2017-07-06 21:51:...|
|jYaJZuli3fVSNDOs6...|   650509405109544|2017-07-02 14:27:...|
|jYaJZuli3fVSNDOs6...|   650509405109544|2017-07-15 03:03:...|
|jYaJZuli3fVSNDOs6...|   650509405109544|2017-07-15 03:03:...|
|jYaJZuli3fVSNDOs6...|   650509405109544|2017-07-01 06:32:...|
|jYaJZuli3fVSNDOs6...|   650509405109544|2017-07-01 10:07:...|
|jYaJZuli3fVSNDOs6...|   650509405109544|2017-07-01 06:07:...|
|jYaJZuli3fVSNDOs6...|   650509405109544|2017-07-15 01:48:...|
|jYaJZuli3fVSNDOs6...|   650509405109544|2017-07-13 03:03:...|
|jYaJZuli3fVSNDOs6...|   650509405109544|2017-07-13 03:03:...|
|JU6Ggudehig6AffZF...|   650509405109544|2017-07-13 14:

### Qual é a população e a densidade demográfica dos bairros onde os concorrentes estão ?

In [1034]:
concorrente.collect()[:2]

[['431962533652067',
  'Boizão Lanches',
  'Bar',
  '2',
  '13190-000 Monte Mor',
  'Monte Mor',
  'SP',
  ''],
 ['1663855903830869',
  'Bar do Serjão',
  'Bar',
  '0',
  'Rua das Dracenas - Americana',
  'Americana',
  'SP',
  '']]

In [1035]:
[concorrente for concorrente in concorrente_categoria if concorrente == "Bar"]

['Bar']

In [1138]:
rdd_concorrente = concorrente.map(lambda x: Row(codigo=x[0], nome=x[1], categoria=x[2], faixa_preco=x[3], 
                                            endereco=x[4], municipio=x[5], uf=x[6], codigo_bairro=x[7]))

In [1139]:
rdd_concorrente.take(3)

[Row(categoria='Bar', codigo='431962533652067', codigo_bairro='', endereco='13190-000 Monte Mor', faixa_preco='2', municipio='Monte Mor', nome='Boizão Lanches', uf='SP'),
 Row(categoria='Bar', codigo='1663855903830869', codigo_bairro='', endereco='Rua das Dracenas - Americana', faixa_preco='0', municipio='Americana', nome='Bar do Serjão', uf='SP'),
 Row(categoria='Restaurant', codigo='567824576564110', codigo_bairro='', endereco='Jarinu', faixa_preco='0', municipio='Jarinu', nome='Recanto Do Kuca', uf='SP')]

In [1140]:
df_concorrente = sqlContext.createDataFrame(rdd_concorrente)

In [1141]:
df_concorrente.registerTempTable("concorrente")

In [1142]:
df_concorrente.count()

4202

In [1143]:
df_eventoFluxo.count()

248589

In [1216]:
df_concorrente_fluxo = sqlContext.sql("select \
                                           nome,categoria,municipio,uf,codigo_bairro \
                                           (select \
                                                 count(ef.codigo_concorrente) qtd_evento \
                                           from \
                                                 evento_fluxo ef \
                                           where \
                                                 c.codigo = ef.codigo_concorrente \
                                           group by \
                                                 ef.codigo_concorrente) qtd_evento \
                                      from \
                                           concorrente c \
                                      ")

ParseException: "\nmismatched input 'count' expecting {')', ','}(line 1, pos 192)\n\n== SQL ==\nselect                                            nome,categoria,municipio,uf,codigo_bairro                                            (select                                                  count(ef.codigo_concorrente) qtd_evento                                            from                                                  evento_fluxo ef                                            where                                                  c.codigo = ef.codigo_concorrente                                            group by                                                  ef.codigo_concorrente) qtd_evento                                       from                                            concorrente c                                       \n---------------------------------------------------------------------------------------------------------------------------------------------------------

In [1205]:
df_concorrente_fluxo.count()

664

In [1206]:
df_concorrente_fluxo.show()

+----------+
|qtd_evento|
+----------+
|       189|
|       861|
|        60|
|      1072|
|       126|
|       267|
|       269|
|         4|
|       205|
|        24|
|       261|
|        33|
|       637|
|        77|
|       187|
|        82|
|       705|
|       174|
|       231|
|       338|
+----------+
only showing top 20 rows

